<a href="https://colab.research.google.com/github/rinongal/stylegan-nada/blob/StyleGAN3-NADA/stylegan3_nada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN3-NADA - The Official Unofficial Notebook

This is an officialy unofficial re-implementation of StyleGAN-NADA using StyleGAN3.
For the implementation used with the StyleGAN-NADA paper please see `stylegan_nada.ipynb` instead.

Please note that the SG3 version of the model has not been thoroughly tested, hyperparameters have not been tuned. There is currently no support for inversions. The notebook is likely to change if and when such tools are released.

Some modifications can cause the StyleGAN3 'grid' to appear in the generated images. I have done my best to mitigate the most glaring causes of this issue, but removing it completely will require further work.

# Step 1: Setup required libraries and models. 
This may take a few minutes.

In [ ]:
#@title Setup
import os

pretrained_model_dir = os.path.join("/content", "models")
os.makedirs(pretrained_model_dir, exist_ok=True)

stylegan_nada_dir = os.path.join("/content", "stylegan_nada")

output_dir = os.path.join("/content", "output")

output_model_dir = os.path.join(output_dir, "models")
output_image_dir = os.path.join(output_dir, "images")

def download_model(model_name):
    download_url = f"https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/{model_name}" 
    output_path = f"{os.path.join(pretrained_model_dir, model_name)}"

    if not os.path.isfile(output_path):
        !wget $download_url -O $output_path
    else:
        print("Model already exists. Skipping download...")
    
# install requirements
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force

!pip install ftfy regex tqdm 
!pip install git+https://github.com/openai/CLIP.git

!git clone -b StyleGAN3-NADA https://github.com/rinongal/stylegan-nada.git $stylegan_nada_dir

from argparse import Namespace

import sys
import numpy as np

from PIL import Image

import torch
import torchvision.transforms as transforms

import copy
import pickle

sys.path.append(stylegan_nada_dir)
sys.path.append(os.path.join(stylegan_nada_dir, "ZSSGAN"))

device = 'cuda'

%load_ext autoreload
%autoreload 2

# Step 2: Choose a model type.

Model will be downloaded.

Re-runs of the cell with the same model will re-use the previously downloaded version. Feel free to experiment and come back to previous models :)

In [ ]:
source_model_type = 'ffhq-t' #@param['ffhq-t', 'ffhq-r', 'metfaces-t', 'metfaces-r', 'afhq-t', 'afhq-r']


model_names = {
    "ffhq-t": "stylegan3-t-ffhq-1024x1024.pkl",
    "ffhq-r": "stylegan3-r-ffhq-1024x1024.pkl",
    "metfaces-t": "stylegan3-t-metfaces-1024x1024.pkl",
    "metfaces-u": "stylegan3-r-metfaces-1024x1024.pkl",
    "afhq-t": "stylegan3-t-afhqv2-512x512.pkl",
    "afhq-r": "stylegan3-r-afhqv2-512x512.pkl"
}

dataset_sizes = {
    "ffhq-t": 1024,
    "ffhq-r": 1024,
    "metfaces-t": 1024,
    "metfaces-u": 1024,
    "afhq-t": 512,
    "afhq-r": 512
}

model_name = model_names[source_model_type]
download_model(model_name)

# Step 3: Train the model.
Describe your source and target class. These describe the direction of change you're trying to apply (e.g. "photo" to "sketch", "photo" to "Fernando Botero painting" or "animal" to "Nicolas Cage").

Alternatively, upload a directory with a small (~3) set of target style images (there is no need to preprocess them in any way) and set `style_image_dir` to point at them. This will use the images as a target rather than the source/class texts.

We find that StyleGAN3 changes typically require 600-800 iterations, and that increasing the batch size to 4 can help mitigate some grid-related artifacts (if you luck out on a GPU that can support the increased batch size).

In [ ]:
from ZSSGAN.model.ZSSGAN import ZSSGAN

import numpy as np

import torch

from tqdm import notebook

from ZSSGAN.utils.file_utils import save_images, get_dir_img_list
from ZSSGAN.utils.training_utils import mixing_noise

from IPython.display import display

source_class = "Photo" #@param {"type": "string"}
target_class = "Sketch" #@param {"type": "string"}

style_image_dir = "" #@param {'type': 'string'}

target_img_list = get_dir_img_list(style_image_dir) if style_image_dir else None

CLIP_model = "ViT-B/32 + ViT-B/16" #@param["ViT-B/32", "ViT-B/16", "ViT-B/32 + ViT-B/16"]

model_choice = ["ViT-B/32", "ViT-B/16"]
model_weights = [0.0, 0.0]

if "ViT-B/32" in CLIP_model:
    model_weights[0] = 1.0
if "ViT-B/16" in CLIP_model:
    model_weights[1] = 1.0

training_iterations = 601 #@param {type: "integer"}
output_interval     = 50 #@param {type: "integer"}
save_interval       = 0 #@param {type: "integer"}

training_args = {
    "size": dataset_sizes[source_model_type],
    "batch": 2,
    "n_sample": 4,
    "output_dir": output_dir,
    "lr": 0.002,
    "frozen_gen_ckpt": os.path.join(pretrained_model_dir, model_name),
    "train_gen_ckpt": os.path.join(pretrained_model_dir, model_name),
    "iter": training_iterations,
    "source_class": source_class,
    "target_class": target_class,
    "lambda_direction": 1.0,
    "lambda_patch": 0.0,
    "lambda_global": 0.0,
    "lambda_texture": 0.0,
    "lambda_manifold": 0.0,
    "auto_layer_k": 0,
    "auto_layer_iters": 0,
    "auto_layer_batch": 8,
    "output_interval": 50,
    "clip_models": model_choice,
    "clip_model_weights": model_weights,
    "mixing": 0.0,
    "phase": None,
    "sample_truncation": 0.7,
    "save_interval": save_interval,
    "target_img_list": target_img_list,
    "img2img_batch": 16,
    "sg3": True,
    "sgxl", False,
}

args = Namespace(**training_args)

print("Loading base models...")
net = ZSSGAN(args)
print("Models loaded! Starting training...")

g_reg_ratio = 4 / 5

g_optim = torch.optim.Adam(
    net.generator_trainable.parameters(),
    lr=args.lr * g_reg_ratio,
    betas=(0 ** g_reg_ratio, 0.99 ** g_reg_ratio),
)

# Set up output directories.
sample_dir = os.path.join(args.output_dir, "sample")
ckpt_dir   = os.path.join(args.output_dir, "checkpoint")

os.makedirs(sample_dir, exist_ok=True)
os.makedirs(ckpt_dir, exist_ok=True)

seed = 3 #@param {"type": "integer"}

torch.manual_seed(seed)
np.random.seed(seed)

# Training loop
fixed_z = torch.randn(args.n_sample, 512, device=device)

for i in notebook.tqdm(range(args.iter)):
    net.train()
        
    sample_z = mixing_noise(args.batch, 512, args.mixing, device)

    [sampled_src, sampled_dst], clip_loss = net(sample_z)

    net.zero_grad()
    clip_loss.backward()

    g_optim.step()

    if i % output_interval == 0:
        net.eval()

        with torch.no_grad():
            [sampled_src, sampled_dst], loss = net([fixed_z], truncation=args.sample_truncation)

            grid_rows = 4

            save_images(sampled_dst, sample_dir, "dst", grid_rows, i)

            img = Image.open(os.path.join(sample_dir, f"dst_{str(i).zfill(6)}.jpg")).resize((1024, 256))
            display(img)
    
    if (args.save_interval > 0) and (i > 0) and (i % args.save_interval == 0):
        snapshot_data = {'G_ema': copy.deepcopy(net.generator_trainable.generator).eval().requires_grad_(False).cpu()}
        snapshot_pkl = f'{ckpt_dir}/{str(i).zfill(6)}.pkl'

        with open(snapshot_pkl, 'wb') as f:
            pickle.dump(snapshot_data, f)

# Step 4: Generate samples with the new model

In [ ]:
truncation = 0.7 #@param {type:"slider", min:0, max:1, step:0.05}

samples = 9

with torch.no_grad():
    net.eval()
    sample_z = torch.randn(samples, 512, device=device)

    [sampled_src, sampled_dst], loss = net([sample_z], truncation=truncation)

    grid_rows = int(samples ** 0.5)

    save_images(sampled_dst, sample_dir, "sampled", grid_rows, 0)

    display(Image.open(os.path.join(sample_dir, f"sampled_{str(0).zfill(6)}.jpg")).resize((768, 768)))